# Importing Modules

The necessary modules are : os, opencv, numpy, tqdm, matplotlib, keras and sklearn

In [0]:
import os
import cv2
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from keras.layers import Input, Conv2D, MaxPooling2D, Conv2DTranspose, concatenate, BatchNormalization, Activation, add
from keras.models import Model, model_from_json
from keras.optimizers import Adam
from keras.layers.advanced_activations import ELU, LeakyReLU
from keras.utils.vis_utils import plot_model
from keras import backend as K 
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

Using TensorFlow backend.


# Constructing Training and Test Datasets

We first load our images from Google Drive, and change directories for future operations.

In [0]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
os.chdir("/content/drive/My Drive/CelebAMaskHQ/")
!ls

## Loading the Images

We first load all the images and the corresponding segmentation masks.

**Kindly store your data in the following format inside your CelebAMaskHQ folder:**
- All the original images in the 'images' folder
- The left eye masks in the 'l_eye' folder
- The right eye masks in the 'r_eye' folder
- The lower lip masks in the 'l_lip' folder
- The upper lip masks in the 'u_lip' folder

For image '0.jpg', the left eye mask should be '0_l_eye.png' and stored in the l_eye folder, the right eye mask should be '0_r_eye.png' and stored in the r_eye folder, the lower lip mask should be '0_l_lip.png' and stored in the l_lip folder, the upper lip mask should be '0_u_lip.png' and stored in the u_lip folder.


In order to format the images in the manner provided, just paste the images in their corresponding folders, and run the “EXTRA : Pre processing of input files” section at the bottom of the notebook. The code snippet will do the rest.

They are stored in two lists X, Y and respectively

Moreover, the images are resized to 256x192


NOTE : In colab, StopIteration errors will occur in the start. The errors are caused by Drive timeouts, and will be handled themselves after repeated running due to increase of data stored in our cache.

In [0]:
img_files = next(os.walk('images/'))[2]
leye_files = next(os.walk('l_eye/'))[2]
reye_files = next(os.walk('r_eye/'))[2]
llip_files = next(os.walk('l_lip/'))[2]
ulip_files = next(os.walk('u_lip/'))[2]

# Used to display the differences between the number of masks available
# For each folder
print(len(img_files))
print(len(leye_files))
print(len(reye_files))
print(len(llip_files))
print(len(ulip_files))

# We create a checkpoint mechanism in order to compensate for
# Google's 12 hour runtime restrictions. It stores the arrays after
# every 2000 iterations, and will load them each time we run our data
try:
    X = np.load('X.npy')
    Y = np.load('Y.npy')
    i = np.load('i.npy')
    count = np.load('count.npy')
except IOError:
    X = []
    Y = []
    i = 0
    count = 0

pbar = tqdm(total = 30000-i)
while(i < 30000):    
    img_f1 = "/content/drive/My Drive/CelebAMaskHQ/images/"+str(i)+".jpg"
    if(os.path.exists(img_f1)):
        print("Image "+img_f1+" exists")
        if(os.path.exists('/content/drive/My Drive/CelebAMaskHQ/l_eye/'+str(i)+'_l_eye.png') and os.path.exists('/content/drive/My Drive/CelebAMaskHQ/r_eye/'+str(i)+'_r_eye.png') and os.path.exists('/content/drive/My Drive/CelebAMaskHQ/l_lip/'+str(i)+'_l_lip.png') and os.path.exists('/content/drive/My Drive/CelebAMaskHQ/u_lip/'+str(i)+'_u_lip.png') ):                
            
            # NOTE : The above if case is made in order to compensate for incomplete data
            # i.e there is a lack of masks for certain images, as noted by the 
            # difference of number of file objects per mask in the dataset

            # Taking main image as input and resizing
            img = cv2.imread('images/'+str(i)+'.jpg', cv2.IMREAD_COLOR)
            resized_img = cv2.resize(img,(256, 192), interpolation = cv2.INTER_CUBIC)
        
            X.append(resized_img)
            # We are reading each and every mask available
            # In accordance to the dataset provided
            leye = cv2.imread('l_eye/{}'.format(str(i)+'_l_eye.png'), cv2.IMREAD_GRAYSCALE)
            reye = cv2.imread('r_eye/{}'.format(str(i)+'_r_eye.png'), cv2.IMREAD_GRAYSCALE)
            llip = cv2.imread('l_lip/{}'.format(str(i)+'_l_lip.png'), cv2.IMREAD_GRAYSCALE)
            ulip = cv2.imread('u_lip/{}'.format(str(i)+'_u_lip.png'), cv2.IMREAD_GRAYSCALE)

            # We resize our data to fit params of MultiResUNet
            resized_leye = cv2.resize(leye,(256, 192), interpolation = cv2.INTER_CUBIC)
            resized_reye = cv2.resize(reye,(256, 192), interpolation = cv2.INTER_CUBIC)
            resized_llip = cv2.resize(llip,(256, 192), interpolation = cv2.INTER_CUBIC)
            resized_ulip = cv2.resize(ulip,(256, 192), interpolation = cv2.INTER_CUBIC)

            # The below statements combine all the four masks into one picture
            # In order to satisfy question requirements(preprocessing)
            op = np.maximum(np.array(resized_leye),np.array(resized_reye))
            op = np.maximum(op, np.array(resized_llip))
            op = np.maximum(op,np.array(resized_ulip))

            count += 1
            Y.append(op)
            pbar.update(1)
            print("Picture "+str(i)+" is valid")

        i += 1
        print("Iteration "+str(i)+" complete")
        
        # Aids in checkpointing data, overwrites after every 2000 iters 
        # To protect against runtime issues
        if(i % 2000 == 0):
            np.save('X',X)
            np.save('Y',Y)
            np.save('i',i)
            np.save('count',count)

print("The number of valid images(images entered) : "+str(count))        

# Saving arrays after completion of loop
np.save('X',X)
np.save('Y',Y)
np.save('i',i)
np.save('count',count)

#Run in order to derive data from original parsing method :

NOTE - In order to compensate for time and handle the parsing process within colab, I had originally divided the dataset into 3 parts, and ran 3 different notebooks simultaneously across the datasets for faster pre processing.

The links to the original (workaround) notebooks are provided below :

https://colab.research.google.com/drive/1xz2f6bkIe7HQG5TAKSxdThYj7DrquULF

https://colab.research.google.com/drive/19Q_rWBzq8Jy7zdJCiz1xIVbaw8JVAyq9

https://colab.research.google.com/drive/13zogbjdIvnSFygFAxuosUsl0yFrgS_Yr

Or, alternatively check the /workaround section of the repository.

**In order to attain the split, just run the notebooks mentioned above, the resulting numpy arrays will be stored in your drive automatically.**


PS: This section may require a high RAM runtime implementation of the notebook. Colab may crash and reallocate resources automatically.
In order to accomodate the size of the dataset, we only use the first 20,000 images extracted.
In order to use the whole extracted dataset, just uncomment the sections involving X3 and Y3

In order to further ease access of resources, do not run the following code and check out the 'improved' sub-section of our 'Define Model, Train, Evaluate' part of the notebook. 

In [0]:
# Loading our arrays
X1 = np.load('X1.npy')
X2 = np.load('X2.npy')
#X3 = np.load('X3.npy')

Y1 = np.load('Y1.npy')
Y2 = np.load('Y2.npy')
#Y3 = np.load('Y3.npy')

# Printing array shapes for confirmation
print(X1.shape)
print(X2.shape)
#print(X3.shape)
print(Y1.shape)
print(Y2.shape)
#print(Y3.shape)

# We use the append function with axis=0 in order to preserve the
# original shape of the arrays
X = np.append(X1,X2,axis=0)
#X = np.append(X,X3,axis=0)

Y = np.append(Y1,Y2,axis=0)
#Y = np.append(Y,Y3,axis=0)

# NOTE : The first dimension of both the arrays should be the same
# Else data may be corrupted/contaminated
print(X.shape)
print(Y.shape)

#Saving arrays for future use
np.save('X',X)
np.save('Y',Y)

## Train-Test Split

The X, Y lists are converted to numpy arrays for convenience. 
Furthermore, the images are divided by 255 to bring down the pixel values to [0...1] range. On the other hand the segmentations masks are converted to binary (0 or 1) values.

Using Sklearn *train_test_split* we split the data randomly into 80% training and 20% testing data

NOTE: Due to colab's constraints, we are using the first ten thousand images only.

In order to use the first twenty thousand processed images, run the above code snippet and comment out the lines involving X3 and Y3 (if they are not commented out already) ,then, for the snippet below, replace 'X1.npy' with 'X.npy' in the first line, and replace 'Y1.npy' with 'Y.npy'

In order to use ALL the processed images:

(if using pre processed data from the parallely ran programs)
Follow the above step and make the changes required in the above code snippet 

Else you simply replace 'X1.npy' with 'X.npy' in the first line, and replace 'Y1.npy' with 'Y.npy'

In [0]:
def LoadData(xloc='X1.npy', yloc='Y1.npy'):
    X = np.load(xloc)
    Y = np.load(yloc)

    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=3)

    Y_train = Y_train.reshape((Y_train.shape[0],Y_train.shape[1],Y_train.shape[2],1))
    Y_test = Y_test.reshape((Y_test.shape[0],Y_test.shape[1],Y_test.shape[2],1))

    X_train = X_train / 255
    X_test = X_test / 255
    Y_train = Y_train / 255
    Y_test = Y_test / 255

    Y_train = np.round(Y_train,0)	
    Y_test = np.round(Y_test,0)	

    print(X_train.shape)
    print(Y_train.shape)
    print(X_test.shape)
    print(Y_test.shape)
    return X_train,Y_train,X_test,Y_test


# MultiResUNet Model

## Model Definition

The MultiResUNet model as described in the [paper](https://arxiv.org/abs/1902.04049) can be found  [here](https://github.com/nibtehaz/MultiResUNet/blob/master/MultiResUNet.py)

In [0]:
def conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(1, 1), activation='relu', name=None):
    '''
    2D Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(1, 1)})
        activation {str} -- activation function (default: {'relu'})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2D(filters, (num_row, num_col), strides=strides, padding=padding, use_bias=False)(x)
    x = BatchNormalization(axis=3, scale=False)(x)

    if(activation == None):
        return x

    x = Activation(activation, name=name)(x)

    return x


def trans_conv2d_bn(x, filters, num_row, num_col, padding='same', strides=(2, 2), name=None):
    '''
    2D Transposed Convolutional layers
    
    Arguments:
        x {keras layer} -- input layer 
        filters {int} -- number of filters
        num_row {int} -- number of rows in filters
        num_col {int} -- number of columns in filters
    
    Keyword Arguments:
        padding {str} -- mode of padding (default: {'same'})
        strides {tuple} -- stride of convolution operation (default: {(2, 2)})
        name {str} -- name of the layer (default: {None})
    
    Returns:
        [keras layer] -- [output layer]
    '''

    x = Conv2DTranspose(filters, (num_row, num_col), strides=strides, padding=padding)(x)
    x = BatchNormalization(axis=3, scale=False)(x)
    
    return x


def MultiResBlock(U, inp, alpha = 1.67):
    '''
    MultiRes Block
    
    Arguments:
        U {int} -- Number of filters in a corrsponding UNet stage
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''

    W = alpha * U

    shortcut = inp

    shortcut = conv2d_bn(shortcut, int(W*0.167) + int(W*0.333) +
                         int(W*0.5), 1, 1, activation=None, padding='same')

    conv3x3 = conv2d_bn(inp, int(W*0.167), 3, 3,
                        activation='relu', padding='same')

    conv5x5 = conv2d_bn(conv3x3, int(W*0.333), 3, 3,
                        activation='relu', padding='same')

    conv7x7 = conv2d_bn(conv5x5, int(W*0.5), 3, 3,
                        activation='relu', padding='same')

    out = concatenate([conv3x3, conv5x5, conv7x7], axis=3)
    out = BatchNormalization(axis=3)(out)

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    return out


def ResPath(filters, length, inp):
    '''
    ResPath
    
    Arguments:
        filters {int} -- [description]
        length {int} -- length of ResPath
        inp {keras layer} -- input layer 
    
    Returns:
        [keras layer] -- [output layer]
    '''


    shortcut = inp
    shortcut = conv2d_bn(shortcut, filters, 1, 1,
                         activation=None, padding='same')

    out = conv2d_bn(inp, filters, 3, 3, activation='relu', padding='same')

    out = add([shortcut, out])
    out = Activation('relu')(out)
    out = BatchNormalization(axis=3)(out)

    for i in range(length-1):

        shortcut = out
        shortcut = conv2d_bn(shortcut, filters, 1, 1,
                             activation=None, padding='same')

        out = conv2d_bn(out, filters, 3, 3, activation='relu', padding='same')

        out = add([shortcut, out])
        out = Activation('relu')(out)
        out = BatchNormalization(axis=3)(out)

    return out


def MultiResUnet(height, width, n_channels):
    '''
    MultiResUNet
    
    Arguments:
        height {int} -- height of image 
        width {int} -- width of image 
        n_channels {int} -- number of channels in image
    
    Returns:
        [keras model] -- MultiResUNet model
    '''


    inputs = Input((height, width, n_channels))

    mresblock1 = MultiResBlock(32, inputs)
    pool1 = MaxPooling2D(pool_size=(2, 2))(mresblock1)
    mresblock1 = ResPath(32, 4, mresblock1)

    mresblock2 = MultiResBlock(32*2, pool1)
    pool2 = MaxPooling2D(pool_size=(2, 2))(mresblock2)
    mresblock2 = ResPath(32*2, 3, mresblock2)

    mresblock3 = MultiResBlock(32*4, pool2)
    pool3 = MaxPooling2D(pool_size=(2, 2))(mresblock3)
    mresblock3 = ResPath(32*4, 2, mresblock3)

    mresblock4 = MultiResBlock(32*8, pool3)
    pool4 = MaxPooling2D(pool_size=(2, 2))(mresblock4)
    mresblock4 = ResPath(32*8, 1, mresblock4)

    mresblock5 = MultiResBlock(32*16, pool4)

    up6 = concatenate([Conv2DTranspose(
        32*8, (2, 2), strides=(2, 2), padding='same')(mresblock5), mresblock4], axis=3)
    mresblock6 = MultiResBlock(32*8, up6)

    up7 = concatenate([Conv2DTranspose(
        32*4, (2, 2), strides=(2, 2), padding='same')(mresblock6), mresblock3], axis=3)
    mresblock7 = MultiResBlock(32*4, up7)

    up8 = concatenate([Conv2DTranspose(
        32*2, (2, 2), strides=(2, 2), padding='same')(mresblock7), mresblock2], axis=3)
    mresblock8 = MultiResBlock(32*2, up8)

    up9 = concatenate([Conv2DTranspose(32, (2, 2), strides=(
        2, 2), padding='same')(mresblock8), mresblock1], axis=3)
    mresblock9 = MultiResBlock(32, up9)

    conv10 = conv2d_bn(mresblock9, 1, 1, 1, activation='sigmoid')
    
    model = Model(inputs=[inputs], outputs=[conv10])

    return model


## Auxiliary Functions

### Custom Metrics

Since Keras does not have build-in support for computing Dice Coefficient or Jaccard Index (at the time of writing), the following functions are declared

In [0]:
def dice_coef(y_true, y_pred):
    smooth = 0.0
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def jacard(y_true, y_pred):

    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum ( y_true_f * y_pred_f)
    union = K.sum ( y_true_f + y_pred_f - y_true_f * y_pred_f)

    return intersection/union

### Saving Model 

Function to save the model

In [0]:
def saveModel(model):

    model_json = model.to_json()

    try:
        os.makedirs('models')
    except:
        pass
    
    fp = open('models/modelP5A.json','w')
    fp.write(model_json)
    model.save_weights('models/modelWA.h5')


### Evaluate the Model

We evaluate the model on test data (X_test, Y_test). 

We compute the values of Jaccard Index and Dice Coeficient, and save the predicted segmentation of first 10 images. The best model is also saved

(This could have been done using keras call-backs as well)

**NOTE : The files saved using this method do not follow the format specified in the task. For proper input and output specifications, kindly refer to the Demonstrations linked at the bottom of this book.**

In [0]:
def evaluateModel(model, X_test, Y_test, batchSize):
    
    try:
        os.makedirs('results')
    except:
        pass 
    

    yp = model.predict(x=X_test, batch_size=batchSize, verbose=1)

    yp = np.round(yp,0)

    for i in range(10):

        plt.figure(figsize=(20,10))
        plt.subplot(1,3,1)
        plt.imshow(np.array(X_test[i]))
        plt.title('Input')
        plt.subplot(1,3,2)
        plt.imshow(Y_test[i].reshape(Y_test[i].shape[0],Y_test[i].shape[1]))
        plt.title('Ground Truth')
        plt.subplot(1,3,3)
        plt.imshow(yp[i].reshape(yp[i].shape[0],yp[i].shape[1]))
        plt.title('Prediction')

        intersection = yp[i].ravel() * Y_test[i].ravel()
        union = yp[i].ravel() + Y_test[i].ravel() - intersection

        jacard = (np.sum(intersection)/np.sum(union))  
        plt.suptitle('Jacard Index'+ str(np.sum(intersection)) +'/'+ str(np.sum(union)) +'='+str(jacard))

        plt.savefig('results/'+str(i)+'fdbl.png',format='png')
        plt.close()


    jacard = 0
    dice = 0
    
    
    for i in range(len(Y_test)):
        yp_2 = yp[i].ravel()
        y2 = Y_test[i].ravel()
        
        intersection = yp_2 * y2
        union = yp_2 + y2 - intersection

        jacard += (np.sum(intersection)/np.sum(union))  

        dice += (2. * np.sum(intersection) ) / (np.sum(yp_2) + np.sum(y2))

    
    jacard /= len(Y_test)
    dice /= len(Y_test)
    


    print('Jacard Index : '+str(jacard))
    print('Dice Coefficient : '+str(dice))
    

    fp = open('models/logfdbl.txt','a')
    fp.write(str(jacard)+'\n')
    fp.close()

    fp = open('models/bestfdbl.txt','r')
    best = fp.read()
    fp.close()

    if(jacard>float(best)):
        print('***********************************************')
        print('Jacard Index improved from '+str(best)+' to '+str(jacard))
        print('***********************************************')
        fp = open('models/bestfdbl.txt','w')
        fp.write(str(jacard))
        fp.close()

        saveModel(model)

### Training the Model

The model is trained and evaluated after each epoch

In [0]:
def trainStep(model, X_train, Y_train, X_test, Y_test, epochs, batchSize):

    
    for epoch in tqdm(range(epochs)):
        print('Epoch : {}'.format(epoch+1))
        model.fit(x=X_train, y=Y_train, batch_size=batchSize, epochs=1, verbose=1)     

        evaluateModel(model,X_test, Y_test,batchSize)

    return model 

## Define Model, Train and Evaluate

### The basic version:
Loads the entire dataset and runs it for the given amount of epochs. 
Kindly change the names of the files as per requirements.

In [0]:

os.chdir("/content/drive/My Drive/CelebAMaskHQ/")
!ls
model = MultiResUnet(height=192, width=256, n_channels=3)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef, jacard, 'accuracy'])

saveModel(model)

fp = open('models/logfdbl.txt','w')
fp.close()
fp = open('models/bestfdbl.txt','w')
fp.write('-1.0')
fp.close()

X_train,Y_train,X_test,Y_test = LoadData('X.npy','Y.npy')

# NOTE : BATCH SIZE MAY NEED TO BE REDUCED IN ORDER FOR IT TO WORK ACROSS DEVICES
trainStep(model, X_train, Y_train, X_test, Y_test, epochs=20, batchSize=10)

### Improved Version 
This section specializes in using the split dataset mentioned above (currently accommodates the 3*10000 split used for training).

It runs each split for the required amount of epochs, then deletes the variables in the memory in order to accomodate certain overload.

In [0]:
#fd - full dataset
os.chdir("/content/drive/My Drive/CelebAMaskHQ/")
!ls
model = MultiResUnet(height=192, width=256, n_channels=3)

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[dice_coef, jacard, 'accuracy'])

saveModel(model)

fp = open('models/logfdbl.txt','w')
fp.close()
fp = open('models/bestfdbl.txt','w')
fp.write('-1.0')
fp.close()

for i in tqdm(range(1,4)):    
    
    print("Using dataset number :"+str(i))
    X_train,Y_train,X_test,Y_test = LoadData('X'+str(i)+'.npy','Y'+str(i)+'.npy')

    # NOTE : BATCH SIZE MAY NEED TO BE REDUCED IN ORDER FOR IT TO WORK ACROSS DEVICES
    trainStep(model, X_train, Y_train, X_test, Y_test, epochs=20, batchSize=10)
    # Done in order to accomodate for RAM overload
    del X_train
    del Y_train
    del X_test
    del Y_test

#EXTRA : Pre processing of input files 
We pre process our input files in order to make it work with our current implementation.

Note : This may cause Google drive timeouts due to the large size of the file, and show an I/O error. In this case just re run the snipppet again.

In [0]:
import os 
  
# Function to rename multiple files 
def renameFiles(folderName): 
      
    for filename in os.listdir(folderName):
        filenamenew = filename.split("_"+folderName)
        ipstring = filenamenew[0]
        opstring = ipstring.lstrip('0')
        if(opstring == ''):
          opstring = '0'
        dst =opstring + "_" + folderName + ".png"
        src =folderName +'/' + filename 
        dst =folderName + '/' + dst 
        print("Original "+src)
        print("Final "+dst)
          
        # rename() function will 
        # rename all the files 
        os.rename(src, dst)
    print("Done")
  
# Driver Code 
if __name__ == '__main__': 
    os.chdir("/content/drive/My Drive/CelebAMaskHQ/")  
    !ls
    # Calling main() function 
    # All completed
    renameFiles("r_eye") 


# Demonstrations:

Kindly check out [MultiResUNetCelebAMaskHQDemo.ipynb](https://colab.research.google.com/drive/10Vb4Ukv4xOG35bS1sUAVp2j2YA2L1xjZ) in order to utilise author built demo networks using the aforementioned architecture.